In [0]:
#IMPORTING DEPENDENCIES
import pandas as pd
import numpy as np

#IMPORTING DATASET
data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/CreditCard_Fraud_Detection/creditcard.csv', index_col=False)
df = data

In [0]:
#ANALYSIS OF BAIS IN DATASET
import matplotlib.pyplot as plt
import seaborn as sns

print('Distribution of the Classes in the subsample dataset')
print(df['Class'].value_counts()/len(df))
sns.countplot('Class', data=df)
plt.title('Unequally Distributed Classes', fontsize=14)
plt.show()

In [0]:
#SINCE THE DATASET IS HIGHLY UNBALANCED - APPLY RESAMPLING AND SHUFFLING TECHNIUES AND THEN ANALYSING THE BAIS IN RESAMPLED AND RESHUFFLED DATA

#STEP 1 - SCALING THE TIME AND AMOUNT ATTRIBUTES USING ROBUSTSCALER 
from sklearn import preprocessing
rob_scalar = preprocessing.RobustScaler()
df['Amt_Scaled'] = rob_scalar.fit_transform(df['Amount'].values.reshape(-1,1))
df['Time_Scaled'] = rob_scalar.fit_transform(df['Time'].values.reshape(-1,1))
df.drop(['Time', 'Amount'], axis=1, inplace=True)

#STEP 2 - SMALL TWEAK IN DATASET - JUST PIVOTING THE POSITIONS OF TIME AND AMOUNT
amt_scl = df['Amt_Scaled']
time_scl = df['Time_Scaled']
df.drop(['Amt_Scaled', 'Time_Scaled'], axis=1, inplace=True)
df.insert(0, 'Amt_Scaled', amt_scl)
df.insert(1, 'Time_Scaled', time_scl)
#df.head()

#RESAMPLING, RESHUFFLING THE DATASET TO CREATE A NEW BALANCED SUB SAMPLE OF THE ORIGINAL DATA
df = df.sample(frac=1)
fraud_n = df.loc[df['Class'] == 1]
no_fraud_n = df.loc[df['Class'] == 0][:492]
normal_dist = pd.concat([fraud_n, no_fraud_n])
df_shuffled = normal_dist.sample(frac=1, random_state=42)
#df_shuffled.head()

In [0]:
#ANALYSING BIAS IN SUB SAMPLE
print('Distribution of the Classes in the subsample dataset')
print(df_shuffled['Class'].value_counts()/len(df_shuffled))
sns.countplot('Class', data=df_shuffled)
plt.title('Equally Distributed Classes', fontsize=14)
plt.show()

In [0]:
#FEATURES AND LABELS
features = df_shuffled
X = features.drop('Class', 1)
Y = df_shuffled.Class

#PREPARING TRAINING DATASET AND TEST DATASET
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=0)

In [0]:
#GAUSSIAN(NB) NAIVE BAYES
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
gaussianNB = GaussianNB()
gaussianNB.fit(X_train, Y_train)
Y_pred_gnb = gaussianNB.predict(X_test)
acc_score_gnb = round(metrics.accuracy_score(Y_test,Y_pred_gnb) * 100)
bal = metrics.balanced_accuracy_score(Y_test,Y_pred_gnb)
pre = metrics.precision_recall_fscore_support(Y_test,Y_pred_gnb, average='binary')
print("Accuracy:", acc_score_gnb, "%")
print("Balanced Accuracy:", round(bal * 100), "%")
print("Precision:", round(pre[0] * 100), "%")
print("Recall:", round(pre[1] * 100), "%")
print("F1-Score:", pre[2])

In [0]:
X_test_cp_gnb = X_test[['Amt_Scaled', 'Time_Scaled']]
#DISPLAYING DECSIONTREECLASSIFIER - DECODED - RESULTS
X_test_cp_gnb['Prediction'] = Y_pred_gnb
X_test_cp_gnb['Prediction'].replace([1,0],['Fraud Transaction','Legit Transaction'],inplace=True)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

X_test_cp_gnb.head()

In [0]:
#K-NEAREST NEIGHBORS(KNN)
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(X_train, Y_train)
Y_pred_knn = knn.predict(X_test)
acc_score_knn = round(metrics.accuracy_score(Y_test,Y_pred_knn) * 100)
bal1 = metrics.balanced_accuracy_score(Y_test,Y_pred_knn)
pre1 = metrics.precision_recall_fscore_support(Y_test,Y_pred_knn, average='binary')
print("Accuracy:", acc_score_knn, "%")
print("Balanced Accuracy:", round(bal1 * 100), "%")
print("Precision:", round(pre1[0] * 100), "%")
print("Recall:", round(pre1[1] * 100), "%")
print("F1-Score:", pre1[2])

In [0]:
X_test_cp_knn = X_test[['Amt_Scaled', 'Time_Scaled']]
#DISPLAYING DECSIONTREECLASSIFIER - DECODED - RESULTS
X_test_cp_knn['Prediction'] = Y_pred_knn
X_test_cp_knn['Prediction'].replace([1,0],['Fraud Transaction','Legit Transaction'],inplace=True)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

X_test_cp_knn.head()

In [0]:
#DECISION TREE CLASSIFIER - UNPRUNDED TREE
from sklearn.tree import DecisionTreeClassifier
dtc_unpruned = DecisionTreeClassifier()
dtc_unpruned.fit(X_train, Y_train)
Y_pred_dtc_unpruned = dtc_unpruned.predict(X_test)
acc_score_dtc_unpruned = round(metrics.accuracy_score(Y_test,Y_pred_dtc_unpruned) * 100)
bal2 = metrics.balanced_accuracy_score(Y_test,Y_pred_dtc_unpruned)
pre2 = metrics.precision_recall_fscore_support(Y_test,Y_pred_dtc_unpruned, average='binary')
print("Accuracy:", acc_score_dtc_unpruned, "%")
print("Balanced Accuracy:", round(bal2 * 100), "%")
print("Precision:", round(pre2[0] * 100), "%")
print("Recall:", round(pre2[1] * 100), "%")
print("F1-Score:", pre2[2])

In [0]:
X_test_cp_dtc_unpruned = X_test[['Amt_Scaled', 'Time_Scaled']]
#DISPLAYING DECSIONTREECLASSIFIER - DECODED - RESULTS
X_test_cp_dtc_unpruned['Prediction'] = Y_pred_dtc_unpruned
X_test_cp_dtc_unpruned['Prediction'].replace([1,0],['Fraud Transaction','Legit Transaction'],inplace=True)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

X_test_cp_dtc_unpruned.head()

In [0]:
#DECISION TREE CLASSIFIER - PRUNDED TREE
dtc_pruned = DecisionTreeClassifier(criterion='entropy', max_depth=3)
dtc_pruned.fit(X_train, Y_train)
Y_pred_dtc_pruned = dtc_pruned.predict(X_test)
acc_score_dtc_pruned = round(metrics.accuracy_score(Y_test,Y_pred_dtc_pruned) * 100)
bal3 = metrics.balanced_accuracy_score(Y_test,Y_pred_dtc_pruned)
pre3 = metrics.precision_recall_fscore_support(Y_test,Y_pred_dtc_pruned, average='binary')
print("Accuracy:", acc_score_dtc_pruned, "%")
print("Balanced Accuracy:", round(bal3 * 100), "%")
print("Precision:", round(pre3[0] * 100), "%")
print("Recall:", round(pre3[1] * 100), "%")
print("F1-Score:", pre3[2])

In [0]:
X_test_cp_dtc_pruned = X_test[['Amt_Scaled', 'Time_Scaled']]
#DISPLAYING DECSIONTREECLASSIFIER - DECODED - RESULTS
X_test_cp_dtc_pruned['Prediction'] = Y_pred_dtc_pruned
X_test_cp_dtc_pruned['Prediction'].replace([1,0],['Fraud Transaction','Legit Transaction'],inplace=True)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

X_test_cp_dtc_pruned.head()

In [0]:
#SUPPORT VECTOR MACHINES - KERNEL : SIGMOID
from sklearn import svm
svm_model_sig = svm.SVC(kernel='sigmoid', gamma='auto')
svm_model_sig.fit(X_train, Y_train)
Y_pred_svm_sig = svm_model_sig.predict(X_test)
acc_score_svm_sig = round(metrics.accuracy_score(Y_test,Y_pred_svm_sig) * 100)
bal4 = metrics.balanced_accuracy_score(Y_test,Y_pred_svm_sig)
pre4 = metrics.precision_recall_fscore_support(Y_test,Y_pred_svm_sig, average='binary')
print("Accuracy:", acc_score_svm_sig, "%")
print("Balanced Accuracy:", round(bal4 * 100), "%")
print("Precision:", round(pre4[0] * 100), "%")
print("Recall:", round(pre4[1] * 100), "%")
print("F1-Score:", pre4[2])

In [0]:
X_test_cp_svm_sig = X_test[['Amt_Scaled', 'Time_Scaled']]
#DISPLAYING DECSIONTREECLASSIFIER - DECODED - RESULTS
X_test_cp_svm_sig['Prediction'] = Y_pred_svm_sig
X_test_cp_svm_sig['Prediction'].replace([1,0],['Fraud Transaction','Legit Transaction'],inplace=True)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

X_test_cp_svm_sig.head()

In [0]:
#SUPPORT VECTOR MACHINES - KERNEL : RBF
svm_model_rbf = svm.SVC(kernel='rbf', gamma='auto')
svm_model_rbf.fit(X_train, Y_train)
Y_pred_svm_rbf = svm_model_rbf.predict(X_test)
acc_score_svm_rbf = round(metrics.accuracy_score(Y_test,Y_pred_svm_rbf) * 100)
bal5 = metrics.balanced_accuracy_score(Y_test,Y_pred_svm_rbf)
pre5 = metrics.precision_recall_fscore_support(Y_test,Y_pred_svm_rbf, average='binary')
print("Accuracy:", acc_score_svm_rbf, "%")
print("Balanced Accuracy:", round(bal5 * 100), "%")
print("Precision:", round(pre5[0] * 100), "%")
print("Recall:", round(pre5[1] * 100), "%")
print("F1-Score:", pre5[2])

In [0]:
X_test_cp_svm_rbf = X_test[['Amt_Scaled', 'Time_Scaled']]
#DISPLAYING DECSIONTREECLASSIFIER - DECODED - RESULTS
X_test_cp_svm_rbf['Prediction'] = Y_pred_svm_rbf
X_test_cp_svm_rbf['Prediction'].replace([1,0],['Fraud Transaction','Legit Transaction'],inplace=True)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

X_test_cp_svm_rbf.head()

In [0]:
#SUPPORT VECTOR MACHINES - KERNEL : LINEAR
from sklearn import svm
from sklearn import metrics
svm_model = svm.SVC(kernel='linear', gamma='auto')
svm_model.fit(X_train, Y_train)
Y_pred_svm = svm_model.predict(X_test)
acc_score_svm = round(metrics.accuracy_score(Y_test,Y_pred_svm) * 100)
bal6 = metrics.balanced_accuracy_score(Y_test,Y_pred_svm)
pre6 = metrics.precision_recall_fscore_support(Y_test,Y_pred_svm, average='binary')
print("Accuracy:", acc_score_svm, "%")
print("Balanced Accuracy:", round(bal6 * 100), "%")
print("Precision:", round(pre6[0] * 100), "%")
print("Recall:", round(pre6[1] * 100), "%")
print("F1-Score:", pre6[2])

In [0]:
X_test_cp_svm = X_test[['Amt_Scaled', 'Time_Scaled']]
#DISPLAYING DECSIONTREECLASSIFIER - DECODED - RESULTS
X_test_cp_svm['Prediction'] = Y_pred_svm
X_test_cp_svm['Prediction'].replace([1,0],['Fraud Transaction','Legit Transaction'],inplace=True)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

X_test_cp_svm.head()